# Best Trajectory Classification Model
This notebook implements the best performing model for trajectory classification, which combines:
1.  **Aggregated Scalar Features**: Min, max, mean, std of position and velocity.
2.  **Physics-based Features**: Coefficients of parabolic fits ($c_2, c_1, c_0$) and residuals (deviation).
3.  **Contextual Features**: Start location clustering and density.

The model used is a **Random Forest Classifier**.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree

# Load Data
print("Loading data...")
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Preprocessing: Time and Sort
for df in [train_df, test_df]:
    df['time_stamp'] = pd.to_datetime(df['time_stamp'], format='mixed')
    df['total_seconds'] = df['time_stamp'].astype('int64')
    
train_df = train_df.sort_values(['traj_ind', 'total_seconds'])
test_df = test_df.sort_values(['traj_ind', 'total_seconds'])

# Calculate Velocity and Speed
def add_kinematics(df):
    dt = df.groupby('traj_ind')['time_stamp'].diff().dt.total_seconds()
    dx = df.groupby('traj_ind')['x'].diff()
    dy = df.groupby('traj_ind')['y'].diff()
    dz = df.groupby('traj_ind')['z'].diff()
    
    df['v_x'] = dx / dt
    df['v_y'] = dy / dt
    df['v_z'] = dz / dt
    df['speed'] = (df['v_x']**2 + df['v_y']**2 + df['v_z']**2)**0.5
    
    # Step distance
    df['step_dist'] = (dx**2 + dy**2 + dz**2)**0.5
    return df

print("Adding kinematic features...")
train_df = add_kinematics(train_df)
test_df = add_kinematics(test_df)

c:\Users\danny\anaconda3\envs\py310\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Loading data...
Adding kinematic features...


In [10]:
# Feature Extraction Functions
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

def extract_trajectory_features(df):
    """Extracts scalar aggregated features."""
    print("Extracting scalar features...")
    grp = df.groupby('traj_ind')
    
    aggs = grp.agg({
        'x': ['first', 'last', 'min', 'max', 'mean', 'std'],
        'y': ['first', 'last', 'min', 'max', 'mean', 'std'],
        'z': ['first', 'last', 'min', 'max', 'mean', 'std'],
        'total_seconds': ['first', 'last', 'count'],
        'speed': ['first', 'mean', 'max', 'std'], # Added first (initial_speed)
        'v_x': [ 'mean', 'std'], # Added first (launch velocity x)
        'v_y': ['mean', 'std'], # Added first (launch velocity y)
        'v_z': [ 'mean', 'std'], # Added first (launch velocity z)
        'step_dist': ['sum']
    })
    
    aggs.columns = ['_'.join(col).strip() for col in aggs.columns.values]
    
    # Datetime features (from traj_starts_df)
    # total_seconds is nanoseconds since epoch
    start_dt = pd.to_datetime(aggs['total_seconds_first'])
    aggs['year'] = start_dt.dt.year
    aggs['month'] = start_dt.dt.month
    aggs['day'] = start_dt.dt.day
    aggs['hour'] = start_dt.dt.hour
    aggs['minute'] = start_dt.dt.minute
    aggs['second'] = start_dt.dt.second
    
    # Derived features
    aggs['duration'] = aggs['total_seconds_last'] - aggs['total_seconds_first']
    aggs['dist_start_end'] = np.sqrt(
        (aggs['x_last'] - aggs['x_first'])**2 + 
        (aggs['y_last'] - aggs['y_first'])**2 + 
        (aggs['z_last'] - aggs['z_first'])**2
    )
    aggs['straightness'] = aggs['dist_start_end'] / aggs['step_dist_sum']
    
    aggs.replace([np.inf, -np.inf], 0, inplace=True)
    aggs = aggs.fillna(0)
    return aggs

def extract_physics_features(df, fit_velocity=False):
    """Extracts parabolic fit coefficients (position) and linear fit parameters (velocity)."""
    print("Extracting physics features (Position & Velocity)...")
    features = []
    indices = []
    
    for traj_id, group in df.groupby('traj_ind'):
        # Prepare time
        t_ns = group['total_seconds'].values
        t = (t_ns - t_ns[0]) / 1e9 # Seconds
        
        # Position data
        x = group['x'].values
        y = group['y'].values
        z = group['z'].values
        
        # Velocity data (handle NaNs from diff)
        vx = group['v_x'].fillna(method='bfill').fillna(0).values
        vy = group['v_y'].fillna(method='bfill').fillna(0).values
        vz = group['v_z'].fillna(method='bfill').fillna(0).values
        
        row_data = {}
        
        # 1. Position Parabolic Fit: p(t) = c2*t^2 + c1*t + c0
        if len(t) < 3:
            # Fallback for tiny trajectories
            for axis in ['x', 'y', 'z']:
                row_data[f'c{axis}_2'] = 0
                row_data[f'c{axis}_1'] = 0
                row_data[f'c{axis}_0'] = group[axis].iloc[0] if len(group) > 0 else 0
                row_data[f'res_{axis}'] = 0
        else:
            try:
                for axis, data in zip(['x', 'y', 'z'], [x, y, z]):
                    c, res, _, _, _ = np.polyfit(t, data, 2, full=True)
                    row_data[f'c{axis}_2'] = c[0]
                    row_data[f'c{axis}_1'] = c[1]
                    row_data[f'c{axis}_0'] = c[2]
                    row_data[f'res_{axis}'] = res[0] if len(res) > 0 else 0.0
            except:
                 for axis in ['x', 'y', 'z']:
                    row_data[f'c{axis}_2'] = 0
                    row_data[f'c{axis}_1'] = 0
                    row_data[f'c{axis}_0'] = 0
                    row_data[f'res_{axis}'] = 0
        
        # 2. Velocity Linear Fit: v(t) = acc*t + v0
        # This captures drag/drift (acc_x, acc_y) and gravity (acc_z) directly from velocity
        if fit_velocity:
            if len(t) < 2:
                for axis in ['x', 'y', 'z']:
                    row_data[f'acc_{axis}'] = 0
                    row_data[f'v0_fit_{axis}'] = 0
            else:
                t_reshaped = t.reshape(-1, 1)
                try:
                    for axis, data in zip(['x', 'y', 'z'], [vx, vy, vz]):
                        model = LinearRegression()
                        model.fit(t_reshaped, data)
                        row_data[f'acc_{axis}'] = model.coef_[0]
                        row_data[f'v0_fit_{axis}'] = model.intercept_
                except:
                    for axis in ['x', 'y', 'z']:
                        row_data[f'acc_{axis}'] = 0
                        row_data[f'v0_fit_{axis}'] = 0

            features.append(row_data)
            indices.append(traj_id)
        
    return pd.DataFrame(features, index=indices)

In [11]:
# Generate Features
# 1. Scalar Features
X_train_scalar = extract_trajectory_features(train_df)
X_test_scalar = extract_trajectory_features(test_df)

# 2. Physics Features
X_train_phys = extract_physics_features(train_df)
X_test_phys = extract_physics_features(test_df)

# 3. Contextual Features (Clustering & Density)
print("Generating contextual features...")
# all_starts = pd.concat([
#     X_train_scalar[['x_first', 'y_first', 'z_first']],
#     X_test_scalar[['x_first', 'y_first', 'z_first']]
# ])

# KMeans Clustering
# kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
# all_starts['start_cluster'] = kmeans.fit_predict(all_starts)

# # Density (KDTree)
# tree = KDTree(all_starts[['x_first', 'y_first', 'z_first']])
# counts = tree.query_radius(all_starts[['x_first', 'y_first', 'z_first']], r=0.1, count_only=True)
# all_starts['start_density'] = counts

# Add back to scalar features
# X_train_scalar['start_cluster'] = all_starts.loc[X_train_scalar.index, 'start_cluster']
# X_test_scalar['start_cluster'] = all_starts.loc[X_test_scalar.index, 'start_cluster']
# X_train_scalar['start_density'] = all_starts.loc[X_train_scalar.index, 'start_density']
# X_test_scalar['start_density'] = all_starts.loc[X_test_scalar.index, 'start_density']

# Combine All Features
print("Combining features...")
X_train_full = pd.concat([X_train_scalar, X_train_phys], axis=1)
X_test_full = pd.concat([X_test_scalar, X_test_phys], axis=1)

# Get Labels
y_train = train_df.groupby('traj_ind')['label'].first().loc[X_train_full.index]

print(f"Final Training Shape: {X_train_full.shape}")
print(f"Final Test Shape: {X_test_full.shape}")

Extracting scalar features...
Extracting scalar features...
Extracting physics features (Position & Velocity)...


C:\Users\danny\AppData\Local\Temp\ipykernel_8616\1922680457.py:64: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vx = group['v_x'].fillna(method='bfill').fillna(0).values
C:\Users\danny\AppData\Local\Temp\ipykernel_8616\1922680457.py:65: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vy = group['v_y'].fillna(method='bfill').fillna(0).values
C:\Users\danny\AppData\Local\Temp\ipykernel_8616\1922680457.py:66: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vz = group['v_z'].fillna(method='bfill').fillna(0).values
c:\Users\danny\anaconda3\envs\py310\lib\site-packages\numpy\lib\polynomial.py:668: RuntimeWarning: invalid value encountered in divide
  lhs /= scale


Extracting physics features (Position & Velocity)...


C:\Users\danny\AppData\Local\Temp\ipykernel_8616\1922680457.py:64: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vx = group['v_x'].fillna(method='bfill').fillna(0).values
C:\Users\danny\AppData\Local\Temp\ipykernel_8616\1922680457.py:65: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vy = group['v_y'].fillna(method='bfill').fillna(0).values
C:\Users\danny\AppData\Local\Temp\ipykernel_8616\1922680457.py:66: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vz = group['v_z'].fillna(method='bfill').fillna(0).values


Generating contextual features...
Combining features...
Final Training Shape: (32741, 41)
Final Test Shape: (8185, 41)


In [12]:
# Model Training and Evaluation
print("Training Random Forest Classifier...")

# Split for validation
X_train, X_val, y_train_split, y_val_split = train_test_split(X_train_full, y_train, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train_split)

# Validation
y_pred = rf_model.predict(X_val)
print("\nValidation Results:")
print(classification_report(y_val_split, y_pred))
print(f"Accuracy: {accuracy_score(y_val_split, y_pred):.4f}")

# Feature Importance
importances = pd.Series(rf_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("\nTop 20 Important Features:")
print(importances.head(20))

Training Random Forest Classifier...

Validation Results:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4472
           1       1.00      1.00      1.00      1589
           2       0.99      0.99      0.99       488

    accuracy                           1.00      6549
   macro avg       1.00      0.99      0.99      6549
weighted avg       1.00      1.00      1.00      6549

Accuracy: 0.9977

Top 20 Important Features:
z_first                0.132730
x_first                0.093963
z_min                  0.090018
z_last                 0.072678
z_mean                 0.057086
x_max                  0.056069
x_min                  0.052206
dist_start_end         0.051907
z_max                  0.031736
y_first                0.030893
z_std                  0.029979
x_mean                 0.029345
x_last                 0.027735
step_dist_sum          0.019188
v_z_mean               0.018922
y_min                  0.018412
stra

In [15]:
# Final Training and Submission
print("Retraining on full dataset...")
rf_final = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf_final.fit(X_train_full, y_train)

print("Predicting on test set...")
y_test_pred = rf_final.predict(X_test_full)

# Create Submission
submission = pd.DataFrame({
    'traj_ind': X_test_full.index,
    'label': y_test_pred
})

output_file = 'submission_best_model.csv'
submission.to_csv(output_file, index=False)
print(f"Submission saved to {output_file}")

Retraining on full dataset...
Predicting on test set...
Submission saved to submission_best_model.csv


In [16]:
X_train_full.columns

Index(['x_first', 'x_last', 'x_min', 'x_max', 'x_mean', 'x_std', 'y_first',
       'y_last', 'y_min', 'y_max', 'y_mean', 'y_std', 'z_first', 'z_last',
       'z_min', 'z_max', 'z_mean', 'z_std', 'total_seconds_first',
       'total_seconds_last', 'total_seconds_count', 'speed_first',
       'speed_mean', 'speed_max', 'speed_std', 'v_x_mean', 'v_x_std',
       'v_y_mean', 'v_y_std', 'v_z_mean', 'v_z_std', 'step_dist_sum', 'year',
       'month', 'day', 'hour', 'minute', 'second', 'duration',
       'dist_start_end', 'straightness'],
      dtype='object')